In [2]:
#前置作業準備，只要一次!!
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

df5k = pd.read_json("paras/mapping5K.json")
dfv = pd.read_csv("volumeData/20230106.csv")
dfv["id"] = dfv["id"].astype("string")
dfv = dfv.drop(columns=['close'])

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

url4 = "https://www.wantgoo.com/investrue/all-alive"
r3 = requests.get(url4, headers = headers).content
soup = BeautifulSoup(r3, "html.parser")
rr3 = soup.prettify()
dfn = pd.read_json(rr3)
dfn = dfn[(dfn["id"]>="1101") & (dfn["id"]<="9999") & (dfn["id"].str.len() == 4)]
dfn = dfn.drop(columns=['type','country','url', 'industries'])
dfn["id"] = dfn["id"].astype("string")

### 計算量比參數 ###
import time
def getRate5K():
    localtime = time.localtime()
    nowTime = time.strftime("%H:%M", localtime)
    Rate5k = df5k[(df5k["e"] >= nowTime) & (df5k["b"] <= nowTime)]["w"].values[0]  #五分K量比預估放大參數 
    print(f"目前時間：{nowTime},預估量放大係數:{Rate5k}")
    return Rate5k

print(getRate5K())

目前時間：09:10,預估量放大係數:10.0
10.0


In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import _beowFmt as fm 

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

r = requests.get("https://www.wantgoo.com/investrue/all-quote-info", headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
df = pd.read_json(rr)
df = df.drop(columns=['tradeDate', 'time','millionAmount'])
df["id"] = df["id"].astype("string")
df = pd.merge(df, dfn, left_on="id", right_on="id")  ## 結合股票名稱
df["float"] = df["close"].astype("float")
df["amp"] = ((df["close"] - df["open"])/df["open"] * 100).round(2)
df["jump"] = df["open"] - df["previousClose"]
df["jumpRate"] = (df["jump"] / df["open"] * 100).round(2)
df = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["id"].str.len() == 4) & (df["close"] > df["previousClose"])].sort_values("jumpRate", ascending=False)
# df.info()
# print(df.head(50))

r2 = requests.get("https://www.wantgoo.com/stock/all-turnover-rates", headers = headers).content
soup = BeautifulSoup(r2, "html.parser")
rr2 = soup.prettify()
dfa = pd.read_json(rr2)
# print(dfa)

dfa["investrueId"] = dfa["investrueId"].astype("string")
dfb = pd.merge(df, dfa, left_on="id", right_on="investrueId")
dfb = dfb.drop(columns=['investrueId'])
dfb = dfb[(dfb["value"] > 1)]
dfb["value"] = dfb["value"].round(2)


dfc = pd.merge(dfb, dfv, left_on="id", right_on="id")
dfc["預估量"] = (dfc["volume"] * getRate5K()).astype(int)        ##預估量
dfc["量比"] = (dfc["預估量"] / dfc["yVolume"]).round(2)     ##量比
dfc["量比周轉"] = (dfc["量比"] + dfc["value"]) / 2
dfc2 = dfc[dfc["value"] > 1]
print(dfc2.loc[:, ["id","market","name","open","close","amp","jump","jumpRate","yVolume","預估量","量比","value","量比周轉"]].sort_values("量比周轉", ascending=False))

ss = ''
for d in dfc2["id"].tolist():
  ss += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態20230109.csv", ss)


######################################################################################################
# import _beowFmt as fm 
# dd = dfb[["id","close","volume"]].sort_values("id")
# ss = 'id,close,yVolume\n'
# for d in dd.values.tolist():
#   ss += f"{d[0]},{d[1]},{d[2]}\n"
# fm.write_LogFile(f"volumeData/20230106.csv", ss)

# todo ##
# https://www.wantgoo.com/investrue/all-previous-weekly-candlestick?v=1672966860000  上一周收盤價


目前時間：20:33,預估量放大係數:1.0
       id    market    name    open   close   amp  jump  jumpRate  yVolume    預估量     量比  value    量比周轉
0    8906       OTC      花王   21.00   20.30 -3.33  1.90      9.05       15    847  56.47   2.49  29.480
4    8171       OTC      天宇   57.30   57.00 -0.52  2.20      3.84    21551  20746   0.96  31.09  16.025
14   4924       OTC   欣厚-KY   15.00   15.70  4.67  0.20      1.33      189   3095  16.38   8.33  12.355
6    3141       OTC      晶宏   83.50   88.60  6.11  2.70      3.23     1349  11420   8.47  15.22  11.845
31   8996    Listed      高力  180.00  183.50  1.94  1.00      0.56    13603  15243   1.12  17.05   9.085
3    6215    Listed      和椿   23.00   24.20  5.22  1.00      4.35     1330   9163   6.89  11.07   8.980
28   3094    Listed      聯傑   24.35   25.95  6.57  0.15      0.62      105   1509  14.37   1.82   8.095
9    4114       OTC      健喬   45.80   44.80 -2.18  1.05      2.29    16704  41696   2.50  12.13   7.315
81   4961    Listed      天鈺  127.00  133.

In [ ]:
import time
localtime = time.localtime()
nowTime = time.strftime("%H:%M", localtime)
print(nowTime)
# multiple_level_data = pd.json_normalize(data, record_path =['Results'], meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], meta_prefix='config_params_', record_prefix='dbscan_')

In [ ]:
import _beowFmt as fm 

df2 = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["volume"] >= 1000)].sort_values(by=['id'], ascending=True)
dd = df2[{"id","volume"}]
ss = ''
for d in dd.values.tolist():
  ss += f"{d[1]}, {d[0]}\n"

fm.write_LogFile(f"keepData/stockVolume.csv", ss)

In [ ]:
while True:
    localtime = time.localtime()
    result = time.strftime("%Y-%m-%d %I:%M:%S %p", localtime)
    print(result)
    time.sleep(1)

In [ ]:
from pathlib import Path  

filepath = Path('https://www.wantgoo.com/investrue/all-quote-info')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

In [ ]:
# importing the requests library
import requests

# api-endpoint
URL = "http://maps.googleapis.com/maps/api/geocode/json"

# location given here
location = "delhi technological university"

# defining a params dict for the parameters to be sent to the API
PARAMS = {'address':location}

# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS)

# extracting data in json format
data = r.json()


# extracting latitude, longitude and formatted address
# of the first matching location
latitude = data['results'][0]['geometry']['location']['lat']
longitude = data['results'][0]['geometry']['location']['lng']
formatted_address = data['results'][0]['formatted_address']

# printing the output
print("Latitude:%s\nLongitude:%s\nFormatted Address:%s"
	%(latitude, longitude,formatted_address))
